# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Specify the path to your CSV file
train_path = '/content/drive/My Drive/dataset/male_players (legacy).csv'
test_path=  '/content/drive/My Drive/dataset/players_22.csv'



In [ ]:
# Load the CSV file into a DataFrame
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)


<ipython-input-20-a06ee600c767>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_path)
<ipython-input-20-a06ee600c767>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(test_path)


# Data Preprocessing


In [ ]:

missing_percentages_train = train_data.isnull().mean() * 100

missing_percentages_test = test_data.isnull().mean() * 100

# Filter columns where missing percentage is 40% or more
columns_with_high_missing1 = missing_percentages_train[missing_percentages_train >= 40].index.tolist()

columns_with_high_missing2 = missing_percentages_test[missing_percentages_test >= 40].index.tolist()


# Print or return the columns with 40% or more missing values
print("Columns with 40% or more missing values:")
print(columns_with_high_missing1)


# Print or return the columns with 40% or more missing values
print("Columns with 40% or more missing values:")
print(columns_with_high_missing2)

Columns with 40% or more missing values:
['club_loaned_from', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'player_tags', 'player_traits', 'goalkeeping_speed']
Columns with 40% or more missing values:
['club_loaned_from', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'player_tags', 'player_traits', 'goalkeeping_speed', 'nation_logo_url']


## Dropping columns

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = "mean")

# Drop columns that are not useful for prediction
columns_to_drop_train = ['player_id', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date',
                   'short_name', 'long_name','player_face_url', 'club_jersey_number','club_loaned_from','club_team_id',
                   'club_joined_date','dob', 'nationality_id', 'nationality_name','nation_jersey_number',
                   'nation_team_id','nation_jersey_number', 'player_tags', 'player_traits','real_face','nation_position']

columns_to_drop_test = ['sofifa_id','player_url', 'short_name', 'long_name',
                   'club_team_id', 'club_joined','dob',
                   'nationality_id', 'nationality_name', 'nation_team_id',
                   'player_tags', 'player_traits', 'player_face_url',
                   'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url','player_face_url','club_loaned_from','real_face','nation_position']


train_data_cleaned = train_data.drop(columns=columns_to_drop_train)
test_data_cleaned = test_data.drop(columns=columns_to_drop_test)

#dropping postions in training data

columns_to_drop_train2 = train_data_cleaned.columns[train_data_cleaned.columns.get_loc('ls'):train_data_cleaned.columns.get_loc('gk') + 1]
train_data_cleaned = train_data_cleaned.drop(columns=columns_to_drop_train2)
test_data_cleaned = test_data_cleaned.drop(columns=columns_to_drop_train2)

# Separate numeric and categorical columns
numeric_features_train = train_data_cleaned.select_dtypes(include=['int64', 'float64']).columns
categorical_features_train = train_data_cleaned.select_dtypes(include=['object']).columns

#grouping ages
train_data_cleaned['age_group'] = pd.cut(train_data_cleaned['age'], bins=[0, 25, 30, 35, 40, 100], labels=['Under 25', '25-30', '30-35', '35-40', 'Over 40'])
train_data_cleaned['age_group'] = train_data_cleaned['age_group'].astype(str)

#grouping ages
test_data_cleaned['age_group'] = pd.cut(train_data_cleaned['age'], bins=[0, 25, 30, 35, 40, 100], labels=['Under 25', '25-30', '30-35', '35-40', 'Over 40'])
test_data_cleaned['age_group'] = train_data_cleaned['age_group'].astype(str)

numeric_features_test = test_data_cleaned.select_dtypes(include=['int64', 'float64']).columns
categorical_features_test = test_data_cleaned.select_dtypes(include=['object']).columns

# Impute missing values for numeric features
numeric_imputer = SimpleImputer(strategy='mean')
train_data_cleaned[numeric_features_train] = numeric_imputer.fit_transform(train_data_cleaned[numeric_features_train])
test_data_cleaned[numeric_features_test] = numeric_imputer.fit_transform(test_data_cleaned[numeric_features_test])


#dropping rows with missing values
train_data_cleaned = train_data_cleaned.dropna()
test_data_cleaned = test_data_cleaned.dropna()



#print(train_data_cleaned.info())
# print(test_data_cleaned.info())


<class 'pandas.core.frame.DataFrame'>
Index: 159810 entries, 0 to 161582
Data columns (total 63 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   player_positions                159810 non-null  object 
 1   overall                         159810 non-null  float64
 2   potential                       159810 non-null  float64
 3   value_eur                       159810 non-null  float64
 4   wage_eur                        159810 non-null  float64
 5   age                             159810 non-null  float64
 6   height_cm                       159810 non-null  float64
 7   weight_kg                       159810 non-null  float64
 8   league_id                       159810 non-null  float64
 9   league_name                     159810 non-null  object 
 10  league_level                    159810 non-null  float64
 11  club_name                       159810 non-null  object 
 12  club_position        

###Encoding non numeric data

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
#function to encode data
def ordinal_encode(data):

    # Get the list of column names with dtype 'object' (categorical features)
    categorical_features_train = data.select_dtypes(include=['object']).columns.tolist()
    # Initialize ordinal encoder
    ordinal_encoder = OrdinalEncoder()

    # Fit and transform the categorical features in the training data
    train_data_cat_encoded = ordinal_encoder.fit_transform(train_data_cleaned[categorical_features_train])


    # Replace original categorical columns with encoded values
    train_data_cleaned[categorical_features_train] = train_data_cat_encoded


    return train_data_cleaned





train_data_cleaned_encoded = ordinal_encode(train_data_cleaned)
test_data_cleaned_encoded = ordinal_encode(test_data_cleaned)

# train_data_cleaned_encoded.info()

# test_data_cleaned_encoded.info()

# test_data_cleaned_encoded.head(5)

In [ ]:
# feature engineering

def feature_engineering(data):
    # Calculate BMI
    data['bmi'] = data['weight_kg'] / ((data['height_cm'] / 100) ** 2)

    # Calculate overall skill rating
    skill_columns = ['skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control']
    data['overall_skill_rating'] = data[skill_columns].mean(axis=1)

    # Calculate overall attacking rating
    attacking_columns = ['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys']
    data['overall_attacking_rating'] = data[attacking_columns].mean(axis=1)

    # Calculate overall movement rating
    movement_columns = ['movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance']
    data['overall_movement_rating'] = data[movement_columns].mean(axis=1)

    # Calculate overall defending rating
    defending_columns = ['defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle']
    data['overall_defending_rating'] = data[defending_columns].mean(axis=1)

    # Calculate overall goalkeeping rating
    goalkeeping_columns = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes']
    data['overall_goalkeeping_rating'] = data[goalkeeping_columns].mean(axis=1)

    # Calculate overall mentality rating
    mentality_columns = ['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision']
    data['overall_mentality_rating'] = data[mentality_columns].mean(axis=1)

    return data

# Apply feature engineering function to datasets
train_data_cleaned = feature_engineering(train_data_cleaned)

test_data_cleaned = feature_engineering(test_data_cleaned)

# train_data_cleaned.info())

# test_data_cleaned.info()
#train_data_cleaned.head()

# Using feature importance to select best features (correlation matrix)

In [ ]:

target_variable = 'overall'

# Compute correlation matrix
corr_matrix = train_data_cleaned.corr()

correlation_with_overall = corr_matrix['overall'].sort_values(ascending=False)

# # Print the top correlations
# print("Top Correlations with Overall Rating:")
# print(correlation_with_overall.head(30))



# Set correlation threshold
correlation_threshold = 0.5


#Select features with correlation higher than the positive threshold
positive_features = corr_matrix[corr_matrix['overall'] >= correlation_threshold].index.tolist()

#Select features with correlation lower than the negative threshold
negative_features = corr_matrix[corr_matrix['overall'] <= -correlation_threshold].index.tolist()

#Combine positive and negative features (excluding 'overall' itself)
relevant_features = list(set(positive_features + negative_features))

# Print the relevant features
print("Relevant Features based on Correlation:")
print(relevant_features)

train_data_final= train_data_cleaned[relevant_features]

test_data_final = test_data_cleaned[relevant_features]
train_data_final.columns


Relevant Features based on Correlation:
['dribbling', 'overall', 'potential', 'mentality_composure', 'passing', 'wage_eur', 'overall_mentality_rating', 'attacking_short_passing', 'value_eur', 'movement_reactions']


Index(['dribbling', 'overall', 'potential', 'mentality_composure', 'passing',
       'wage_eur', 'overall_mentality_rating', 'attacking_short_passing',
       'value_eur', 'movement_reactions'],
      dtype='object')

# Model selection and Tuning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pickle
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


In [92]:
#training
# Splitting data into features (X) and target variable (y)
X = train_data_final.drop(columns=['overall'])  # Features
y = train_data_final['overall']  # Target variable

# Splitting data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)




#### Linear Regression

In [85]:
#linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the model
lr_model = LinearRegression()

# Fit the model on the training data
lr_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred_lr = lr_model.predict(X_val)

# Calculate RMSE on the validation set
lr_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_lr))
print(f"Linear Regression Validation RMSE: {lr_val_rmse}")


Linear Regression Validation RMSE: 2.8967782557457977


#### Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the model
dt_model = DecisionTreeRegressor(random_state=42)

# Fit the model on the training data
dt_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred_dt = dt_model.predict(X_val)

# Calculate RMSE on the validation set
dt_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_dt))
print(f"Decision Tree Regressor Validation RMSE: {dt_val_rmse}")


Decision Tree Regressor Validation RMSE: 1.9584791959333072


#### Random Forest

In [ ]:
#random forest regressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error



rf_model = RandomForestRegressor(random_state=42)

# Cross-validation training and evaluation
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rf_rmse_scores = np.sqrt(-rf_scores)

# Fit the model on the entire training data
rf_model.fit(X_train, y_train)

# Predictions on validation set
y_val_pred_rf = rf_model.predict(X_val)

# Calculate RMSE on validation set
rf_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_rf))
print(f"RandomForestRegressor Validation RMSE: {rf_val_rmse}")




KeyboardInterrupt: 

#### XG Boost

In [ ]:
from xgboost import XGBRegressor
# Initialize XGBoostRegressor
xgb_model = XGBRegressor(random_state=42)

# Cross-validation training and evaluation
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
xgb_rmse_scores = np.sqrt(-xgb_scores)

# Fit the model on the entire training data
xgb_model.fit(X_train, y_train)

# Predictions on validation set
y_val_pred_xgb = xgb_model.predict(X_val)

# Calculate RMSE on validation set
xgb_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_xgb))
print(f"XGBoostRegressor Validation RMSE: {xgb_val_rmse}")

# Hypertuning Random Forest and XG Boost

#### Random forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pickle
import joblib

# Define the parameter grid
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_distributions,
                                   n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)
# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# Train the model with tuned parameters
best_rf_model = random_search.best_estimator_

# Cross-validation training and evaluation with the best parameters
rf_scores = cross_val_score(best_rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_rmse_scores = np.sqrt(-rf_scores)
print(f"Cross-validation RMSE scores: {rf_rmse_scores}")
print(f"Average cross-validation RMSE: {np.mean(rf_rmse_scores)}")

# Fit the model on the entire training data with the best parameters
best_rf_model.fit(X_train, y_train)

# Predictions on validation set
y_val_pred_rf = best_rf_model.predict(X_val)

# Calculate RMSE on validation set
rf_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_rf))
print(f"RandomForestRegressor Validation RMSE with best parameters: {rf_val_rmse}")

# Save the tuned model to a file
filename = '/content/drive/My Drive/dataset/rf_model.pkl'
joblib.dump(best_rf_model, filename)
print(f"Saved best model to {filename}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [97]:
X_train.columns

Index(['dribbling', 'potential', 'mentality_composure', 'passing', 'wage_eur',
       'overall_mentality_rating', 'attacking_short_passing', 'value_eur',
       'movement_reactions'],
      dtype='object')

#### XG Boost

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

# Initialize the XGBoost regressor
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)

# Fit GridSearchCV to the data
grid_search.fit(X_train, y_train)

# View the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best RMSE: {np.sqrt(-grid_search.best_score_)}")

# Train the final model using the best parameters
best_xgb_model = grid_search.best_estimator_

# Fit the model on the entire training data
best_xgb_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred_xgb = best_xgb_model.predict(X_val)

# Calculate RMSE on the validation set
xgb_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_xgb))
print(f"XGBoost Regressor Validation RMSE: {xgb_val_rmse}")

# Save the tuned model to a file
filename = '/content/drive/My Drive/dataset/xgb_model.pkl'
pickle.dump(best_xgb_model, open(filename, 'wb'))
print(f"Saved best model to {filename}")


#### Linear Hypertuning

In [86]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for alpha
param_grid = {'alpha': [0.1, 1, 10, 100]}

# Initialize the Ridge model
ridge = Ridge()

# Initialize GridSearchCV
ridge_grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid,
                                 scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)

# Fit GridSearchCV to the data
ridge_grid_search.fit(X_train, y_train)

# View the best parameters and the best score
print(f"Best parameters for Ridge: {ridge_grid_search.best_params_}")
print(f"Best RMSE for Ridge: {np.sqrt(-ridge_grid_search.best_score_)}")

# Train the final Ridge model using the best parameters
best_ridge_model = ridge_grid_search.best_estimator_

# Fit the model on the entire training data
best_ridge_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred_ridge = best_ridge_model.predict(X_val)

# Calculate RMSE on the validation set
ridge_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_ridge))
print(f"Ridge Regression Validation RMSE: {ridge_val_rmse}")

# Save the tuned model to a file
filename = '/content/drive/My Drive/dataset/linear_model.pkl'
pickle.dump(best_ridge_model, open(filename, 'wb'))
print(f"Saved best model to {filename}")


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters for Ridge: {'alpha': 0.1}
Best RMSE for Ridge: 2.903181617646342
Ridge Regression Validation RMSE: 2.896778255606053
Saved best model to /content/drive/My Drive/dataset/linear_model.pkl


Decision Tree Optimization

In [73]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the model
dt = DecisionTreeRegressor(random_state=42)

# Initialize GridSearchCV
dt_grid_search = GridSearchCV(estimator=dt, param_grid=param_grid,
                              scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)

# Fit GridSearchCV to the data
dt_grid_search.fit(X_train, y_train)

# View the best parameters and the best score
print(f"Best parameters for Decision Tree: {dt_grid_search.best_params_}")
print(f"Best RMSE for Decision Tree: {np.sqrt(-dt_grid_search.best_score_)}")

# Train the final model using the best parameters
best_dt_model = dt_grid_search.best_estimator_

# Fit the model on the entire training data
best_dt_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred_best_dt = best_dt_model.predict(X_val)

# Calculate RMSE on the validation set
best_dt_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_best_dt))
print(f"Better Decision Tree Validation RMSE: {best_dt_val_rmse}")





Fitting 5 folds for each of 162 candidates, totalling 810 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


Best parameters for Decision Tree: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10}
Best RMSE for Decision Tree: 1.75421677307553


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


Better Decision Tree Validation RMSE: 1.7124487617497988


In [87]:
# Save the tuned model to a file
filename = '/content/drive/My Drive/dataset/dt_model.pkl'
pickle.dump(best_dt_model, open(filename, 'wb'))
print(f"Saved best model to {filename}")

Saved best model to /content/drive/My Drive/dataset/dt_model.pkl


Save as a pickle file

In [ ]:

from google.colab import drive

# Save the model to a file
filename = 'rf_model.pkl'
pickle.dump(rf_model, open(filename, 'wb'))
print(f"Saved model to {filename}")

# Testing


#XGB Boost Test

In [84]:
import xgboost as xgb
import pickle
from sklearn.metrics import mean_squared_error

# Load the saved model
loaded_model = pickle.load(open('/content/drive/My Drive/dataset/xgb_model.pkl', 'rb'))


# Select relevant features for prediction using .loc
X_predict = test_data_final.loc[:, relevant_features]


desired_order = ['potential', 'passing', 'overall_mentality_rating', 'attacking_short_passing', 'wage_eur', 'value_eur', 'mentality_composure', 'movement_reactions', 'dribbling']



X_predict = test_data_final.drop(columns=['overall'])  # Features
y_predict = test_data_final['overall']  # Target variable


# Rearrange columns in test_data_final according to desired_order
X_predict = X_predict.loc[:, desired_order]

# Predict on the new dataset
y_pred = loaded_model.predict(X_predict)


# Calculate RMSE (Root Mean Squared Error)
test_rmse = np.sqrt(mean_squared_error(y_predict, y_pred))
print(f"Test RMSE: {test_rmse}")


Test RMSE: 1.3558535985061781


In [99]:
X_predict.columns

Index(['potential', 'passing', 'overall_mentality_rating',
       'attacking_short_passing', 'wage_eur', 'value_eur',
       'mentality_composure', 'movement_reactions', 'dribbling'],
      dtype='object')

# Linear Regression

In [103]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# Load the trained linear regression model
linear_regression_model = joblib.load('/content/drive/My Drive/dataset/linear_model.pkl')


y_test = test_data_final['overall']

test_data_final_reordered = test_data_final.drop(columns=['overall'], errors='ignore')

# Predict ratings for test data
y_pred = linear_regression_model.predict(test_data_final_reordered)


# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Linear Regression Test RMSE: {rmse}")


Linear Regression Test RMSE: 2.7323602043769153


In [106]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error


decision_tree_model = joblib.load('/content/drive/My Drive/dataset/dt_model.pkl')


y_test = test_data_final['overall']

# Drop the 'overall' column from the test data
test_data_final_reordered = test_data_final.drop(columns=['overall'])


# Predict ratings for test data
y_pred = decision_tree_model.predict(test_data_final_reordered)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Decision Tree Test RMSE: {rmse}")


Decision Tree Test RMSE: 1.230949311099096


##### Decision Tree was the best model however, XGB Boost was the best ensemble moder, linear regression performed poorly compared to the others